In [42]:
# import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [43]:
udp_2017 = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/udp_2017results.csv')
vars_wdummies_1314 = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/demog_with_cra_dummies_201314.csv')
vars_wdummies_1516 = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/demog_with_cra_dummies_20151617.csv')
vars_wdummies_09 = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/demog_with_cra_dummies_2009.csv')
len(vars_wdummies_1314)


949

#### Get Loan Data

In [44]:
parsed_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data_1/'
files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2015", "2016", "2017"]]
dfs = []

for file in files:
    df_lol = []
    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]

        df_lol.append(row_list)
        
                
all_20151617 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2015-2017", "% Loans to MHI borrowers"])
    


files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2013", "2014"]]
dfs = []
for file in files:
    df_lol = []

    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]
        df_lol.append(row_list)
                
all_201314 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2013-2014", "% Loans to MHI borrowers"])



    
files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2009"]]
df_lol = []
dfs = []
for file in files:
    df_lol = []
    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]
        df_lol.append(row_list)
                
all_2009 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2009", "% Loans to MHI borrowers"])
all_2009.head()


,Geoid,Total loans 2009,% Loans to MHI borrowers
0,6001400100,55,0.981818
1,6001400200,20,1.000000
2,6001400300,64,0.906250
3,6001400400,48,0.979167
4,6001400500,57,0.877193


In [45]:
all_2009.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/2009_loans.csv", index=False)
all_201314.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/201314_loans.csv", index=False)
all_20151617.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/20151617_loans.csv", index=False)

In [46]:
def encode_udp(row):
    if (row == 'MHI - Advanced Gentrification'):
        return 4
    elif (row == 'LI - Ongoing Gentrification and/or Displacement'):
        return 3     
    elif (row == 'LI - At Risk of Gentrification and/or Displacement'):
        return 2   
    elif (row == 'MHI - Not Losing Low Income Households' or row == 'LI - Not Losing Low Income Households'):
        return 1
    else:
        return np.nan #this includes all of the exclusion typologies, college town, or unreliable data


#### 2009

In [47]:
udp_merge_09 = pd.merge(vars_wdummies_09, udp_2017, how='left', right_on='geo_fips', left_on='Geoid') #demog with UDP
udp_merge_09_loans = pd.merge(udp_merge_09, all_2009, how='left', on='Geoid')#demog with udp and loans

print("09 vars: ", len(vars_wdummies_09))
print("udp: ", len(udp_2017))
print("loans tract: ", len(all_2009)) 
print("merged 1: ", len(udp_merge_09))
print("merged 2: ", len(udp_merge_09_loans))
# rec1_udp_loans_merge.head()

09 vars:  976
udp:  2131
loans tract:  1378
merged 1:  976
merged 2:  976


In [48]:
udp_merge_09_loans["UDP code"] = udp_merge_09_loans["Typology"].apply(lambda x: encode_udp(x))
udp_merge_09_loans = udp_merge_09_loans.dropna() #drop any tracts that don't have one of the prescribed typologies
udp_merge_09_loans.head()

,Geoid,Percent of population 25 years and over with Bachelor's degree,% below poverty level,% Single Family,% Owner Occupied,Percent NH White alone,Percent NH Black or African African alone,Percent NH Asian alone,Percent Hispanic,Total number of housing units,...,County_San Mateo County,County_Santa Clara County,County_Solano County,County_Sonoma County,geo_fips,Typology,old disp type,Total loans 2009,% Loans to MHI borrowers,UDP code
1,6001400300,0.321,0.069,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,...,0,0,0,0,6.001400e+09,LI - Not Losing Low Income Households,2.0,64,0.906250,1.0
2,6001400400,0.440,0.040,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,...,0,0,0,0,6.001400e+09,MHI - Advanced Gentrification,6.0,48,0.979167,4.0
3,6001400500,0.281,0.060,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,...,0,0,0,0,6.001400e+09,LI - Ongoing Gentrification and/or Displacement,4.0,57,0.877193,3.0
4,6001400600,0.519,0.264,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,...,0,0,0,0,6.001401e+09,LI - Ongoing Gentrification and/or Displacement,4.0,31,0.870968,3.0
5,6001400700,0.194,0.081,0.572421,0.395335,0.356832,0.453986,0.095104,0.049218,2016,...,0,0,0,0,6.001401e+09,LI - At Risk of Gentrification and/or Displace...,4.0,119,0.714286,2.0


In [49]:
x_09 =  udp_merge_09_loans[["CRA",'% Loans to MHI borrowers', 'Percent of population 25 years and over with Bachelor\'s degree',
       '% below poverty level', '% Single Family', 'Percent NH White alone', 'Total number of housing units', 'Median home value', 'Median income',
       'Total Loans', 'County_San Mateo County',
    'County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County']]
y_09 =  udp_merge_09_loans["UDP code"]

x_09_constant = sm.add_constant(x_09)
x_09_constant
r = sm.regression.linear_model.OLS(y_09, x_09_constant)
results_r = r.fit()
print(results_r.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/udp_regressions/2009_udp_linear.html', 'w') as outfile:
    outfile.write(results_r.summary().as_html())

                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     8.870
Date:                Mon, 15 Apr 2019   Prob (F-statistic):           8.32e-22
Time:                        00:42:06   Log-Likelihood:                -881.85
No. Observations:                 674   AIC:                             1802.
Df Residuals:                     655   BIC:                             1887.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

#### 2013-2014

In [50]:
udp_merge_1314 = pd.merge(vars_wdummies_1314, udp_2017, how='left', right_on='geo_fips', left_on='Geoid')
udp_loans_merge_1314 = pd.merge(udp_merge_1314, all_201314, how='left', on='Geoid')

print("rec1vars: ", len(vars_wdummies_1314))
print("udp: ", len(udp_2017))
print("loans tract: ", len(all_201314)) 
print("merged 1: ", len(udp_merge_1314))
print("merged 2: ", len(udp_loans_merge_1314))
# rec1_udp_loans_merge.head()

rec1vars:  949
udp:  2131
loans tract:  1549
merged 1:  949
merged 2:  949


In [51]:
udp_loans_merge_1314["UDP code"] = udp_loans_merge_1314["Typology"].apply(lambda x: encode_udp(x))
udp_loans_merge_1314 = udp_loans_merge_1314.dropna() #drop any tracts that don't have one of the prescribed typologies
# udp_loans_merge_1314.head()

In [52]:
x_1314 =  udp_loans_merge_1314[["CRA",'% Loans to MHI borrowers', 'Percent of population 25 years and over with Bachelor\'s degree',
       '% below poverty level', '% Single Family', 'Percent NH White alone', 'Total number of housing units', 'Median home value', 'Median income',
       'Total Loans', 'County_San Mateo County',
    'County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County']]
y_1314 =  udp_loans_merge_1314["UDP code"]

x_1314_constant = sm.add_constant(x_1314)
x_1314_constant
r = sm.regression.linear_model.OLS(y_1314, x_1314_constant)
results_r = r.fit()
print(results_r.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/udp_regressions/201314_udp.html', 'w') as outfile:
    outfile.write(results_r.summary().as_html())

                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.217
Method:                 Least Squares   F-statistic:                     12.38
Date:                Mon, 15 Apr 2019   Prob (F-statistic):           5.63e-32
Time:                        00:42:06   Log-Likelihood:                -960.12
No. Observations:                 739   AIC:                             1958.
Df Residuals:                     720   BIC:                             2046.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [59]:
x_1314_nomedinc =  udp_loans_merge_1314[["CRA",'% Loans to MHI borrowers', 'Percent of population 25 years and over with Bachelor\'s degree',
       '% below poverty level', '% Single Family', 'Percent NH White alone', 'Total number of housing units', 'Median home value', 
       'Total Loans', 'County_San Mateo County',
    'County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County']]
y_1314_nomedinc =  udp_loans_merge_1314["UDP code"]

x_1314_nomedinc_c = sm.add_constant(x_1314_nomedinc)
rm = sm.regression.linear_model.OLS(y_1314_nomedinc, x_1314_nomedinc_c)
results_rm = rm.fit()
print(results_rm.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/udp_regressions/201314_udp_nomedinc.html', 'w') as outfile:
    outfile.write(results_r.summary().as_html())

                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     13.08
Date:                Mon, 15 Apr 2019   Prob (F-statistic):           1.96e-32
Time:                        00:44:42   Log-Likelihood:                -960.40
No. Observations:                 739   AIC:                             1957.
Df Residuals:                     721   BIC:                             2040.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [53]:
vif = pd.DataFrame()
no_counties = x_1314.loc[:, ~x_1314.columns.isin(["County_Alameda County", "County_Contra Costa County", "County_Marin County", 
          "County_San Francisco County", "County_San Mateo County", "County_Santa Clara County",
              "County_Solano County", "County_Sonoma County", "% Owner Occupied"])]
vif["VIF Factor"] = [variance_inflation_factor(no_counties.values, i) for i in range(no_counties.shape[1])]
vif["features"] = no_counties.columns
vif.round(2)


vif.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/collinearity/VIF_201314_udp.csv")


In [61]:
vif = pd.DataFrame()
no_counties = x_1314_nomedinc.loc[:, ~x_1314_nomedinc.columns.isin(["County_Alameda County", "County_Contra Costa County", "County_Marin County", 
          "County_San Francisco County", "County_San Mateo County", "County_Santa Clara County",
              "County_Solano County", "County_Sonoma County", "% Owner Occupied"])]
vif["VIF Factor"] = [variance_inflation_factor(no_counties.values, i) for i in range(no_counties.shape[1])]
vif["features"] = no_counties.columns
vif.round(2)


# vif.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/collinearity/VIF_201314_udp_nomedinc.csv")


,VIF Factor,features
0,2.97,CRA
1,43.07,% Loans to MHI borrowers
2,15.61,Percent of population 25 years and over with B...
3,3.15,% below poverty level
4,6.17,% Single Family
5,6.39,Percent NH White alone
6,11.09,Total number of housing units
7,28.91,Median home value
8,3.57,Total Loans


#### 2015-2017 Models

In [54]:
udp_merge_151617 = pd.merge(vars_wdummies_1516, udp_2017, how='left', right_on='geo_fips', left_on='Geoid') #demog with UDP
udp_merge_151617_loans = pd.merge(udp_merge_151617, all_20151617, how='left', on='Geoid')#demog with udp and loans
udp_merge_151617_loans["UDP code"] = udp_merge_151617_loans["Typology"].apply(lambda x: encode_udp(x))
udp_merge_151617_loans = udp_merge_151617_loans.dropna() #drop any tracts that don't have one of the prescribed typologies
udp_merge_151617_loans.head()



,Geoid,Percent of population 25 years and over with Bachelor's degree,% below poverty level,% Single Family,% Owner Occupied,Percent NH White alone,Percent NH Black or African African alone,Percent NH Asian alone,Percent Hispanic,Total number of housing units,...,County_San Mateo County,County_Santa Clara County,County_Solano County,County_Sonoma County,geo_fips,Typology,old disp type,Total loans 2015-2017,% Loans to MHI borrowers,UDP code
1,6001400300,0.321,0.069,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,...,0,0,0,0,6001400300,LI - Not Losing Low Income Households,2.0,140,1.000000,1.0
2,6001400400,0.440,0.040,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,...,0,0,0,0,6001400400,MHI - Advanced Gentrification,6.0,112,0.991071,4.0
3,6001400500,0.281,0.060,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,...,0,0,0,0,6001400500,LI - Ongoing Gentrification and/or Displacement,4.0,134,0.947761,3.0
4,6001400600,0.519,0.264,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,...,0,0,0,0,6001400600,LI - Ongoing Gentrification and/or Displacement,4.0,66,1.000000,3.0
5,6001400700,0.194,0.081,0.572421,0.395335,0.356832,0.453986,0.095104,0.049218,2016,...,0,0,0,0,6001400700,LI - At Risk of Gentrification and/or Displace...,4.0,222,0.968468,2.0


In [55]:
x_151617 =  udp_merge_151617_loans[["CRA", "% Loans to MHI borrowers", "% Single Family",
     "% below poverty level", "Percent NH White alone", "Total number of housing units",
     "Percent of population 25 years and over with Bachelor's degree", "Median home value", "Median income",
            'County_San Mateo County','County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County']]
y_151617 =  udp_merge_151617_loans["UDP code"]
x_151617_constant = sm.add_constant(x_151617)
r = sm.regression.linear_model.OLS(y_151617, x_151617_constant)
results_r = r.fit()
print(results_r.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/udp_regressions/20151617_udp_all.html', 'w') as outfile:
    outfile.write(results_r.summary().as_html())


                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     13.35
Date:                Mon, 15 Apr 2019   Prob (F-statistic):           3.96e-33
Time:                        00:42:06   Log-Likelihood:                -957.20
No. Observations:                 738   AIC:                             1950.
Df Residuals:                     720   BIC:                             2033.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [56]:
vif = pd.DataFrame()
no_counties = x_151617.loc[:, ~x_151617.columns.isin(["County_Alameda County", "County_Contra Costa County", "County_Marin County", 
          "County_San Francisco County", "County_San Mateo County", "County_Santa Clara County",
              "County_Solano County", "County_Sonoma County", "% Owner Occupied"])]
vif["VIF Factor"] = [variance_inflation_factor(no_counties.values, i) for i in range(no_counties.shape[1])]
vif["features"] = no_counties.columns
vif.round(2)
vif.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/collinearity/VIF_20151617_udp.csv")
vif.head()



,VIF Factor,features
0,3.239095,CRA
1,48.675366,% Loans to MHI borrowers
2,10.644317,% Single Family
3,3.556788,% below poverty level
4,6.134200,Percent NH White alone
